## RAG-Based Q&A for FNMA Selling Guide 
 
### Advanced RAG Approaches: Query Transformation --Stepback Prompting

### Advanced RAG Approaches: Retrieval Enhancement -- Fusion Retrieval/Hybrid Search

> *This notebook should work well with the **`Amazon Bedrock and LangChain freamwork`** kernel in SageMaker Studio*

### Use Case of this POC
#### Purpose
To help answer questions about selling loans to Fannie Mae through leveraging the RAG architecture.

The LLM will try to answer from the documents in easy language.

#### Dataset
Fannie Mae Selling Guide (PDF document)




### Advanced RAG Framework

<img src="./images/Adv_RAG_Framework.jpg" width="700" height="500">

### Query Transformation --Stepback Prompting
This technique can be combined with regular rag by doing retrieval on both the original and step-back question.

<img src="./images/stepback RAG architecture.jpg" width="600" height="400">

### Retrieval Enhancement -- Fusion Retrieval/Hybrid Search

A relatively old idea that you could take the best from both worlds — keyword-based old school search — sparse retrieval algorithms like tf-idf or search industry standard BM25 — and modern semantic or vector search and combine it in one retrieval result.

<img src="./images/fusion-retrieval.jpg" width="600" height="600">

## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V1 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: FAISS available through LangChain

  In this notebook we are using this in-memory vector-store to store both the embeddings and the documents. In an enterprise context this could be replaced with a persistent store such as AWS OpenSearch, RDS Postgres with pgVector, ChromaDB, Pinecone or Weaviate.


In [ ]:
# !pip install langchainhub

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-text-express-v1")`

Check Available text generation and embedding models Ids under Amazon Bedrock.


In [4]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock


# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':1024, 'temperature':0.1,'top_p':0.5})
#llm = Bedrock(model_id="meta.llama2-13b-chat-v1", client=boto3_bedrock, model_kwargs={'temperature':0.1,'top_p':0.5, 'max_gen_len':1024})

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

In [5]:
# - create the Anthropic Model
llm_gen = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':1024, 'temperature':0,'top_p':0.5})
#llm_gen =  Bedrock(model_id="meta.llama2-13b-chat-v1", client=boto3_bedrock, model_kwargs={'temperature':0,'top_p':0.5, 'max_gen_len':1024})


## Data Preparation
Let's first transform files to build the document store and vector index. For this example we will be using public FNMA Selling Guidedocuments from

Leverage [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8192 tokens, which roughly translates to ~32,000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [6]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# loader = PyPDFDirectoryLoader("./data/")

# documents = loader.load()
import pickle

with open('./data/loaded_document.pkl', 'rb') as file:
    documents = pickle.load(file)

In [7]:
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size =1250,
    chunk_overlap  = 125,
)
docs = text_splitter.split_documents(documents)

In [8]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 1199 documents loaded is 2087 characters.
After the split we have 2720 documents more than the original 1199.
Average length among 2720 documents (after split) is 969 characters.


### Stepback Prompting 
How to implement a stepback prompting in LangChain?

In [9]:

from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

######################################

# Few Shot Examples
examples = [
    {
        "input": "Can part-time income be used to qualify for the mortgage?",
        "output": "what kind of incomes can qualify the borrower for the mortgage?"
    },
    {
        "input": "Jan Sindel’s was born in what country?", 
        "output": "what is Jan Sindel’s personal history?"
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

######################################

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
    # Few shot examples
    few_shot_prompt,
    # New question
    ("user", "{question}"),
])

######################################

question_gen = prompt | llm_gen | StrOutputParser()

######################################

question = "Can be a home for the mortgage be powered by solar panels only?"

######################################

question_gen.invoke({"question": question})

' What are the energy source requirements for a home to qualify for a mortgage?'

In [10]:
question_gen.invoke({"question": 'Can part-time income be used to qualify?'})

' What types of income can be used to qualify for a mortgage?'

In [11]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS,Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

# if 'vectorstore_faiss ' in globals(): # If you've already made your vectordb this will delete it so you start fresh
#     vectorstore_faiss .delete_collection()
    
# vectorstore_faiss = FAISS.from_documents(
#      docs,
#     bedrock_embeddings,
# )

## embedding the documents and save to a vector store

if 'vectordb' in globals(): # If you've already made your vectordb this will delete it so you start fresh
    vectordb.delete_collection()
    
vectordb = Chroma.from_documents(documents=docs, embedding=bedrock_embeddings)


In [16]:
# build the index 
retriever_vector =vectordb.as_retriever(
        search_type="similarity", search_kwargs={"k":4})

In [15]:
# initialize the bm25 retriever based on traditional way

from langchain.retrievers import  BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(
    documents=docs, k=4
)

In [17]:
# initialize the ensemble retriever
retriever_ensemble = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever_vector], weights=[0.4, 0.6]
)

## Question Answering

Now that we have our vector store in place, we can start asking questions.

In [12]:
query = "What are acceptable flood insurance policies for the lender?"

Now we have the relevant documents, it's time to use the LLM to generate an answer based on these documents. 


LangChain provides an abstraction of how this can be done easily.

### Quick way
You have the possibility to use the wrapper provided by LangChain which wraps around the Vector Store and takes input the LLM.
This wrapper performs the following steps behind the scences:
- Take the question as input
- Create question embedding
- Fetch relevant documents
- Stuff the documents and the question into a prompt
- Invoke the model with the prompt and generate the answer in a human readable manner.

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{normal_context}
</context

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["normal_context", "question"]
)

In [14]:
from langchain import hub

response_prompt = hub.pull("langchain-ai/stepback-answer")

response_prompt.template = 'Use the following pieces of context to provide a concise answer to the question at the end. \
If you do not know the answer, just say that you do not know, do not try to make up an answer. \
\n\n{normal_context}\n{step_back_context}\n\nOriginal Question: {question}\nAnswer:'


In [18]:
qa_chain = {
    # Retrieve context using the normal question
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever_ensemble,
    # Retrieve context using the step-back question
    "step_back_context": question_gen | retriever_ensemble,
    # Pass on the question
    "question": lambda x: x["question"]
} | response_prompt | llm | StrOutputParser()


In [19]:
answer = qa_chain.invoke({"question":query})
print_ww(answer )

 Based on the provided context, acceptable flood insurance policies for the lender include:

- A standard policy issued under the National Flood Insurance Program (NFIP).

- A policy issued by a private insurer, provided the terms and amount of coverage are at least equal
to that provided under an NFIP policy. The private insurer must also meet Fannie Mae's rating
requirements.

- A Policy Declaration page is acceptable evidence of flood insurance.

The flood insurance policy must include the standard mortgagee clause naming the lender as the
mortgagee and providing that any action or failure to act by the borrower or owner of the insured
property will not invalidate the interest of the mortgagee.


Let's ask a different question:

In [22]:
query_2 =  'When can rental income be used to qualify?'
#query_3 = 'What are lender incentives for mortgage borrowers?'

In [23]:
answer_2 = qa_chain.invoke({"question":query_2})
print_ww(answer_2)

 Based on the provided documents, rental income can be used to qualify the borrower in the following
situations:

- If the rental income is derived from the subject property, the property must be a two- to four-
unit principal residence property in which the borrower occupies one of the units, or a one- to
four-unit investment property.

- If the income is derived from a property that is not the subject property, there are no
restrictions on the property type. Rental income from commercial properties owned by the borrower is
acceptable.

- Boarder income from one-unit properties can be used up to 30% of the total gross income used to
qualify the borrower, if certain criteria are met such as the boarder has lived with the borrower
for the past 12 months.

- For HomeReady loans, rental income from accessory units is allowed with standard documentation.

- The lender must verify and document the rental income used for qualifying purposes based on the
requirements outlined in the Selling G

In [20]:
query_3 = 'Can part-time income be used to qualify?'

In [21]:
answer_3 = qa_chain.invoke({"question":query_3})
print_ww(answer_3)

 Based on the provided context, here are the key points for using part-time income to qualify:

- Part-time and secondary employment income can be used to qualify if it has been received for at
least 12 months and is likely to continue.

- Income that has been received for a shorter period (but no less than 12 months) may be considered
if there are positive factors to reasonably offset the shorter income history.

- There can be no gaps in employment greater than one month in the most recent 12-month period when
using part-time income to qualify.

- Income from part-time employment that does not meet the minimum 12-month history may still be
considered under other standard income guidelines, such as interest and dividend income.

- The lender must verify and document part-time income per the standard documentation requirements
based on the type of income.

In summary, yes part-time income can be used to qualify for a mortgage if it meets the requirements
around documentation, history, 